In [13]:
#41で定義した関数を使用するためにimport
from importnb import imports
with imports('ipynb'):
    import knock41 
sentences = knock41.morphs_chunk()

from itertools import combinations
def path_to_root(now,sentence):                                #文節番号nowから根までのパスを求める(返り値は文節番号のリスト)
    chunk = sentence[now]                                      #現在の文節を取得
    path = []                                                  #出力用配列の初期化
    if chunk.dst != -1:                                        #現在の文節に係り先があるとき
        path.append(now)                                       #現在の文節番号をpathに追加
        nxt = chunk.dst                                        #現在の文節の係り先を取得
        while nxt != -1:                                       #係り先がなくなるまで
            path.append(nxt)                                   #現在の文節の係り先をpathに追加
            nxt = sentence[nxt].dst                            #係り先の更新
    return path

sentence = sentences[1]                                        #出力対象となる文を設定
nouns = []                                                     #名詞を含む文節番号を保存する配列

for i,chunk in enumerate(sentence):                            #文節番号と文節を取得
    if '名詞' in [morph.pos for morph in chunk.morphs]:        #文節の中に名詞が含まれているとき
        nouns.append(i)                                        #nounに文節番号を追加

for i,j in combinations(nouns,2):                                 #全組み合わせを取得
    pathx = path_to_root(i,sentence)                              #xから根までのパス
    pathy = path_to_root(j,sentence)                              #yから根までのパス
    path1 = []                                                    #文節iから文節jのパス or 文節iから文節kに至る直前のパス
    path2 = []                                                    #文節jから文節kに至る直前のパス
    path3 = []                                                    #文節kの内容
    flg = 0                                                       #pathxとpathyの交点を判定するフラグ(交わった後は同じ経路をたどるため、交点は別で管理)

    #パターン１(文節iから構造木の根に至る経路上に文節jが存在する場合)
    if j in pathx:                                                                                                            #文節iから構造木の根に至る経路上に文節jが存在する場合
        for k in pathx:                                                                                                       #pathx上の任意の文節番号kについて
            if k == i:                                                                                                        #kとiが等しいとき
                morphs = 'X' + ''.join([word.surface if word.pos == '助詞' else '' for word in sentence[k].morphs])           #その文節(文節番号i)での名詞を'X'に置換
                path1.append(''.join(morphs))                                                                                 #それをpath1に追加
            elif k != j:                                                                                                      #kがjと等しくないとき(i->k->j)
                morphs = ''.join([word.surface if word.pos != '記号' else '' for word in sentence[k].morphs])                 #記号以外の単語をmorphsに追加
                path1.append(''.join(morphs))                                                                                 #それをpath1に追加
            else:                                                                                                             #kとjが等しいとき
                morphs = 'Y' + ''.join([word.surface if word.pos =='助詞' else '' for word in sentence[k].morphs])            #その文節(文節番号j)での名詞を'Y'に置換
                path1.append(''.join(morphs))                                                                                 #それをpath1に追加
                break
        print(' -> '.join(path1))                                                                                             #表示

    #パターン２(文節iと文節jから構造木の根に至る経路上で共通の文節kで交わる場合)
    else:
        for k in pathx:                                                                                                       #文節番号iから交点までの探索
            if k == i:                                                                                                        #iとkが等しいとき
                morphs = 'X' + ''.join([word.surface if word.pos == '助詞' else '' for word in sentence[k].morphs])           #その文節(文節番号i)での名詞を'X'に置換
                path1.append(''.join(morphs))                                                                                 #それをpath1に追加                                                                               
            elif k not in pathy:                                                                                              #文節番号kがpathyとの交点でないとき           
                morphs = ''.join([word.surface if word.pos != '記号' else '' for word in sentence[k].morphs])                 #記号以外の単語をmorphsに追加
                path1.append(''.join(morphs))                                                                                 #それをpath1に追加
            else:
                break
        for l in pathy:                                                                                                       #文節番号jから交点までの探索
            if l == j:                                                                                                        #jとlが等しいとき
                morphs = 'Y' + ''.join([word.surface if word.pos == '助詞' else '' for word in sentence[l].morphs])           #その文節(文節番号j)での名詞を'Y'に置換
                path2.append(''.join(morphs))                                                                                 #それをpath2に追加
            elif l != k:                                                                                                      #文節番号kがpathyとの交点でないとき
                morphs = ''.join([word.surface if word.pos != '記号' else '' for word in sentence[l].morphs])                 #記号以外の単語をmorphsに追加
                path2.append(''.join(morphs))                                                                                 #それをpath2に追加
            elif l == k and flg == 0:                                                                                         #pathxとpathyの交点になったとき
                flg = 1                                                                                                       #フラグを立てる
                morphs = ''.join([word.surface if word.pos != '記号' else '' for word in sentence[l].morphs])                 #記号以外の単語をmorphsに追加
                path3.append(''.join(morphs))                                                                                 #それをpath2に追加      
        print(' -> '.join(path1) + ' | ' + ' -> '.join(path2) + ' | ' + ' -> '.join(path3))                                   #表示



X | Yのう -> 研究分野とも -> される | 語
X | Y -> エーアイとは -> 研究分野とも -> される | 語
X | Yとは -> 研究分野とも -> される | 語
X | Yという -> コンピュータ（）という -> 道具を -> 用いて -> 研究する -> 計算機科学（）の -> 一分野を -> 指す -> 研究分野とも -> される | 語
X | Yと -> コンピュータ（）という -> 道具を -> 用いて -> 研究する -> 計算機科学（）の -> 一分野を -> 指す -> 研究分野とも -> される | 語
X | Yという -> 道具を -> 用いて -> 研究する -> 計算機科学（）の -> 一分野を -> 指す -> 研究分野とも -> される | 語
X | Yを -> 用いて -> 研究する -> 計算機科学（）の -> 一分野を -> 指す -> 研究分野とも -> される | 語
X | Yを -> 研究する -> 計算機科学（）の -> 一分野を -> 指す -> 研究分野とも -> される | 語
X | Y -> 計算機科学（）の -> 一分野を -> 指す -> 研究分野とも -> される | 語
X | Yの -> 一分野を -> 指す -> 研究分野とも -> される | 語
X | Yを -> 指す -> 研究分野とも -> される | 語
X -> Y
X -> 語 | Yの -> 推論 -> 知的行動を -> 代わって -> 行わせる -> 技術または -> される | 研究分野とも
X -> 語 | Yや -> 推論 -> 知的行動を -> 代わって -> 行わせる -> 技術または -> される | 研究分野とも
X -> 語 | Y -> 知的行動を -> 代わって -> 行わせる -> 技術または -> される | 研究分野とも
X -> 語 | Yなどの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> される | 研究分野とも
X -> 語 | Yを -> 代わって -> 行わせる -> 技術または -> される | 研究分野とも
X -> 語 | Yに -> 代わって -> 行わせる -> 技術または -> される | 研究分野とも
X -> 語 | 